In [ ]:
from turtle import width
import torch
from PIL import Image
from torch.utils.data import DataLoader
import numpy as np

In [ ]:

class ImagePixelDatasetHigherDimention(torch.utils.data.Dataset):

  def __init__(self,image_path,L):
    self.image = np.array(Image.open(image_path))
    self.height,self.width,_ = self.image.shape
    self.L = L


  def __len__(self):
    return self.height * self.width

  def __getitem__(self, idx):

    y = idx // self.width
    x = idx % self.width
    color = self.image[y,x]/255

    y = np.interp(y,[0,self.height],[-1,1])
    x = np.interp(x,[0,self.width],[-1,1])

    input = []

    for l in range(0,self.L):
        input.append(np.sin(2**l * np.pi * x))
        input.append(np.cos(2**l * np.pi * x))

    for l in range(0,self.L):
        input.append(np.sin(2**l * np.pi * y))
        input.append(np.cos(2**l * np.pi * y))


    return np.array(input), color

In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

path = "/content/drive/My Drive/sifnos-greece-3840x2160-12799.jpg"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

L=3

dataset = ImagePixelDatasetHigherDimention(path,L)
dataloader = DataLoader(dataset,batch_size=16,shuffle=True)
train_feature, train_labels = next(iter(dataloader))
print(train_feature, train_labels)

tensor([[-0.8032,  0.5957, -0.9569, -0.2903,  0.5556, -0.8315, -0.7566, -0.6539,
          0.9894, -0.1449, -0.2868, -0.9580],
        [ 0.9988,  0.0491,  0.0980, -0.9952, -0.1951,  0.9808, -0.1908, -0.9816,
          0.3746,  0.9272,  0.6947,  0.7193],
        [ 0.7625,  0.6470,  0.9866, -0.1629, -0.3214, -0.9469, -0.7735, -0.6338,
          0.9805, -0.1965, -0.3854, -0.9228],
        [-0.9771, -0.2127,  0.4157, -0.9095, -0.7561,  0.6544, -0.9026, -0.4305,
          0.7771, -0.6293, -0.9781, -0.2079],
        [ 0.1710,  0.9853,  0.3369,  0.9415,  0.6344,  0.7730,  0.3611, -0.9325,
         -0.6734,  0.7392, -0.9957,  0.0929],
        [ 0.9376, -0.3477, -0.6519, -0.7583,  0.9887,  0.1500,  0.8587, -0.5125,
         -0.8802, -0.4746,  0.8355, -0.5495],
        [-0.9437,  0.3307, -0.6242, -0.7812,  0.9753,  0.2207, -0.9811,  0.1937,
         -0.3800, -0.9250,  0.7030,  0.7112],
        [-0.5127, -0.8586,  0.8804,  0.4743,  0.8351, -0.5501, -0.1507,  0.9886,
         -0.2979,  0.9546, -0.

In [ ]:
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torchvision import transforms


class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1,hidden_size2,hidden_size3, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(hidden_size3, output_size)
        self.relu4 = nn.ReLU()


    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.relu3(self.fc3(x))
        x = self.relu4(self.fc4(x))
        return x

In [ ]:

total_size = len(dataset)
train_size = int(0.8 * total_size)
test_size = total_size - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [ ]:
batch_size = 1024
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import time

start_time = time.time()

input_size = 4*L # Define your input size based on the number of pixels in an image
output_size = 3  # Assuming RGB prediction


model_hd_LBFGS = MLP(input_size,256,128,64, output_size)


criterion = nn.MSELoss()
#optimizer = torch.optim.LBFGS(model_hd_LBFGS.parameters(), lr=1,max_eval=30,max_iter=3)
optimizer = torch.optim.LBFGS(model_hd_LBFGS.parameters(), lr=0.01, max_iter=20, max_eval=None, tolerance_grad=1e-5, tolerance_change=1e-9, history_size=100)


def closure():
    optimizer.zero_grad()
    outputs = model_hd_LBFGS(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    return loss


num_epochs =1
for epoch in range(num_epochs):
    for inputs, targets in train_dataloader:
        # Flatten the inputs if needed


        inputs = torch.tensor(inputs,dtype=torch.float32)
        targets = torch.tensor(targets,dtype=torch.float32)



        optimizer.step(closure)

    # Print the loss at each iteration
    print(f'Epoch {epoch + 1}, Loss: {closure().item()}')

print("---Duration: %s seconds ---" % (time.time() - start_time))

<ipython-input-18-cf5e2d0730dc>:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs,dtype=torch.float32)
<ipython-input-18-cf5e2d0730dc>:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets,dtype=torch.float32)


Epoch 1, Loss: 0.09403299540281296
---Duration: 8925.825008869171 seconds ---


In [ ]:
model_hd_LBFGS.eval()
with torch.no_grad():
    test_loss = 0.0
    for inputs, targets in test_dataloader:
        # Flatten the inputs if needed

        inputs = torch.tensor(inputs,dtype=torch.float32)
        targets = torch.tensor(targets,dtype=torch.float32)

        outputs = model_hd_LBFGS(inputs)

        test_loss += criterion(outputs, targets)

    average_test_loss = test_loss / len(test_dataloader)
    print(f'Test Loss: {average_test_loss:.4f}')

<ipython-input-19-090a907e8146>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs,dtype=torch.float32)
<ipython-input-19-090a907e8146>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets,dtype=torch.float32)


Test Loss: 0.1025


In [ ]:
torch.save(model_hd_LBFGS, '/content/drive/My Drive/model_mlp_hd_LBFGS.pt')

In [ ]:
coor = [dataset[i][0] for i in range(len(dataset))]
coor = torch.tensor(coor)

model_hd_LBFGS.eval()

batch_size = 64
with torch.no_grad():
  pre = []
  for i in range(0, coor.size(0), batch_size):
    batch_coor = torch.tensor(coor[i:i+batch_size, :],dtype=torch.float32)
    batch_pred = model_hd_LBFGS(batch_coor)
    pre.append(batch_pred)

predicted_rgb = torch.cat(pre, dim=0)

predicted_rgb = predicted_rgb.view(dataset.height, dataset.width, 3).numpy() * 255
predicted_image = Image.fromarray(predicted_rgb.astype('uint8'))

predicted_image.save('./mlp_hd_LBFGS.png')

<ipython-input-21-e04e76a9ead6>:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  coor = torch.tensor(coor)
<ipython-input-21-e04e76a9ead6>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_coor = torch.tensor(coor[i:i+batch_size, :],dtype=torch.float32)


In [ ]:
from turtle import width
import torch
from PIL import Image
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from tqdm import tqdm


In [ ]:

class ImagePixelDatasetHigherDimention(torch.utils.data.Dataset):

  def __init__(self,image_path,L):
    self.image = np.array(Image.open(image_path))
    self.height,self.width,_ = self.image.shape
    self.L = L


  def __len__(self):
    return self.height * self.width

  def __getitem__(self, idx):

    y = idx // self.width
    x = idx % self.width
    color = self.image[y,x]/255

    y = np.interp(y,[0,self.height],[-1,1])
    x = np.interp(x,[0,self.width],[-1,1])

    input = []

    for l in range(0,self.L):
        input.append(np.sin(2**l * np.pi * x))
        input.append(np.cos(2**l * np.pi * x))

    for l in range(0,self.L):
        input.append(np.sin(2**l * np.pi * y))
        input.append(np.cos(2**l * np.pi * y))


    return np.array(input), color

In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

path = "/content/drive/My Drive/sifnos-greece-3840x2160-12799.jpg"

Mounted at /content/drive


In [ ]:

L=3

dataset = ImagePixelDatasetHigherDimention(path,L)
dataloader = DataLoader(dataset,batch_size=16,shuffle=True)
train_feature, train_labels = next(iter(dataloader))
print(train_feature, train_labels)

tensor([[ 0.8594, -0.5113, -0.8788, -0.4772,  0.8387, -0.5446, -0.6626,  0.7490,
         -0.9925,  0.1219, -0.2419, -0.9703],
        [ 0.8865,  0.4627,  0.8204, -0.5718, -0.9382, -0.3461,  0.2560, -0.9667,
         -0.4950,  0.8689, -0.8601,  0.5100],
        [ 0.0442,  0.9990,  0.0882,  0.9961,  0.1758,  0.9844, -0.7604, -0.6494,
          0.9877, -0.1564, -0.3090, -0.9511],
        [-0.0866,  0.9962, -0.1726,  0.9850, -0.3400,  0.9404,  0.4924, -0.8704,
         -0.8572,  0.5150, -0.8829, -0.4695],
        [ 0.9974,  0.0719,  0.1435, -0.9897, -0.2840,  0.9588, -0.9918,  0.1276,
         -0.2532, -0.9674,  0.4899,  0.8718],
        [-0.3751, -0.9270,  0.6954,  0.7186,  0.9995,  0.0327,  0.1851, -0.9827,
         -0.3638,  0.9315, -0.6777,  0.7353],
        [-0.8241, -0.5664,  0.9336, -0.3584, -0.6691, -0.7431, -0.5373, -0.8434,
          0.9063,  0.4226,  0.7660, -0.6428],
        [-0.8138,  0.5811, -0.9459, -0.3245,  0.6139, -0.7894, -0.9877,  0.1564,
         -0.3090, -0.9511,  0.

In [ ]:

import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torchvision import transforms


class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.relu2 = nn.ReLU()

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        return x

class MLP3(nn.Module):
    def __init__(self, input_size, hidden_size1,hidden_size2, output_size):
        super(MLP3, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, output_size)
        self.relu3 = nn.ReLU()

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.relu3(self.fc3(x))
        return x

class MLP4(nn.Module):
    def __init__(self, input_size, hidden_size1,hidden_size2,hidden_size3, output_size):
        super(MLP4, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(hidden_size3, output_size)
        self.relu4 = nn.ReLU()

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.relu3(self.fc3(x))
        x = self.relu4(self.fc4(x))
        return x

In [ ]:
total_size = len(dataset)
train_size = int(0.8 * total_size)
test_size = total_size - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
batch_size = 4096
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import time

input_size = 4*L # Define your input size based on the number of pixels in an image
output_size = 3  # Assuming RGB prediction

model_256_LBFGS = MLP(input_size, 256, output_size)


start_time = time.time()


criterion = nn.MSELoss()
#optimizer = torch.optim.LBFGS(model_hd_LBFGS.parameters(), lr=1,max_eval=30,max_iter=3)
optimizer = torch.optim.LBFGS(model_256_LBFGS.parameters(), lr=0.01, max_iter=20, max_eval=None, tolerance_grad=1e-5, tolerance_change=1e-9, history_size=100)


def closure():
    optimizer.zero_grad()
    outputs = model_256_LBFGS(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    return loss


num_epochs =3
for epoch in tqdm(range(num_epochs)):
    for inputs, targets in train_dataloader:
        # Flatten the inputs if needed


        inputs = torch.tensor(inputs,dtype=torch.float32)
        targets = torch.tensor(targets,dtype=torch.float32)



        optimizer.step(closure)

    # Print the loss at each iteration
    print(f'Epoch {epoch + 1}, Loss: {closure().item()}')

print("---Duration: %s seconds ---" % (time.time() - start_time))

  0%|          | 0/3 [00:00<?, ?it/s]<ipython-input-11-a652445eab4f>:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs,dtype=torch.float32)
<ipython-input-11-a652445eab4f>:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets,dtype=torch.float32)
 33%|███▎      | 1/3 [07:47<15:34, 467.42s/it]

Epoch 1, Loss: 0.02762218751013279


 67%|██████▋   | 2/3 [15:22<07:40, 460.33s/it]

Epoch 2, Loss: 0.026545308530330658


100%|██████████| 3/3 [22:58<00:00, 459.63s/it]

Epoch 3, Loss: 0.024592235684394836
---Duration: 1378.901659488678 seconds ---


In [ ]:
model_256_LBFGS.eval()
with torch.no_grad():
    test_loss = 0.0
    for inputs, targets in test_dataloader:
        # Flatten the inputs if needed

        inputs = torch.tensor(inputs,dtype=torch.float32)
        targets = torch.tensor(targets,dtype=torch.float32)

        outputs = model_256_LBFGS(inputs)

        test_loss += criterion(outputs, targets)

    average_test_loss = test_loss / len(test_dataloader)
    print(f'Test Loss: {average_test_loss:.4f}')

<ipython-input-12-49818b9c1ea8>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs,dtype=torch.float32)
<ipython-input-12-49818b9c1ea8>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets,dtype=torch.float32)


Test Loss: 0.0263


In [ ]:
torch.save(model_256_LBFGS, '/content/drive/My Drive/model_mlp_256_LBFGS.pt')

In [ ]:
coor = [dataset[i][0] for i in range(len(dataset))]
coor = torch.tensor(coor)

model_256_LBFGS.eval()

batch_size = 64
with torch.no_grad():
  pre = []
  for i in range(0, coor.size(0), batch_size):
    batch_coor = torch.tensor(coor[i:i+batch_size, :],dtype=torch.float32)
    batch_pred = model_256_LBFGS(batch_coor)
    pre.append(batch_pred)

predicted_rgb = torch.cat(pre, dim=0)

predicted_rgb = predicted_rgb.view(dataset.height, dataset.width, 3).numpy() * 255
predicted_image = Image.fromarray(predicted_rgb.astype('uint8'))

predicted_image.save('./mlp_256_LBFGS.png')

<ipython-input-14-57afc6420671>:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  coor = torch.tensor(coor)
<ipython-input-14-57afc6420671>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_coor = torch.tensor(coor[i:i+batch_size, :],dtype=torch.float32)


In [ ]:
import time

batch_size = 4096
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

input_size = 4*L # Define your input size based on the number of pixels in an image
output_size = 3  # Assuming RGB prediction

model_256_128_LBFGS = MLP3(input_size, 256,128, output_size)


start_time = time.time()


criterion = nn.MSELoss()
#optimizer = torch.optim.LBFGS(model_hd_LBFGS.parameters(), lr=1,max_eval=30,max_iter=3)
optimizer = torch.optim.LBFGS(model_256_128_LBFGS.parameters(), lr=0.01, max_iter=20, max_eval=None, tolerance_grad=1e-5, tolerance_change=1e-9, history_size=100)


def closure():
    optimizer.zero_grad()
    outputs = model_256_128_LBFGS(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    return loss


num_epochs =3
for epoch in tqdm(range(num_epochs)):
    for inputs, targets in train_dataloader:
        # Flatten the inputs if needed


        inputs = torch.tensor(inputs,dtype=torch.float32)
        targets = torch.tensor(targets,dtype=torch.float32)



        optimizer.step(closure)

    # Print the loss at each iteration
    print(f'Epoch {epoch + 1}, Loss: {closure().item()}')

print("---Duration: %s seconds ---" % (time.time() - start_time))

  0%|          | 0/3 [00:00<?, ?it/s]<ipython-input-23-67d1607c9789>:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs,dtype=torch.float32)
<ipython-input-23-67d1607c9789>:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets,dtype=torch.float32)
 33%|███▎      | 1/3 [16:13<32:27, 973.90s/it]

Epoch 1, Loss: 0.02550884522497654


 67%|██████▋   | 2/3 [32:46<16:25, 985.15s/it]

Epoch 2, Loss: 0.02555813454091549


100%|██████████| 3/3 [49:08<00:00, 982.67s/it]

Epoch 3, Loss: 0.023976227268576622
---Duration: 2948.0328373908997 seconds ---


In [ ]:
model_256_128_LBFGS.eval()
with torch.no_grad():
    test_loss = 0.0
    for inputs, targets in test_dataloader:
        # Flatten the inputs if needed

        inputs = torch.tensor(inputs,dtype=torch.float32)
        targets = torch.tensor(targets,dtype=torch.float32)

        outputs = model_256_128_LBFGS(inputs)

        test_loss += criterion(outputs, targets)

    average_test_loss = test_loss / len(test_dataloader)
    print(f'Test Loss: {average_test_loss:.4f}')

<ipython-input-24-65d67874c9db>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs,dtype=torch.float32)
<ipython-input-24-65d67874c9db>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets,dtype=torch.float32)


Test Loss: 0.0252


In [ ]:
torch.save(model_256_128_LBFGS, '/content/drive/My Drive/model_mlp_256_128_LBFGS.pt')

In [ ]:
coor = [dataset[i][0] for i in range(len(dataset))]
coor = torch.tensor(coor)

model_256_128_LBFGS.eval()

batch_size = 64
with torch.no_grad():
  pre = []
  for i in range(0, coor.size(0), batch_size):
    batch_coor = torch.tensor(coor[i:i+batch_size, :],dtype=torch.float32)
    batch_pred = model_256_128_LBFGS(batch_coor)
    pre.append(batch_pred)

predicted_rgb = torch.cat(pre, dim=0)

predicted_rgb = predicted_rgb.view(dataset.height, dataset.width, 3).numpy() * 255
predicted_image = Image.fromarray(predicted_rgb.astype('uint8'))

predicted_image.save('./mlp_256_128_LBFGS.png')

<ipython-input-26-d2577d591004>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_coor = torch.tensor(coor[i:i+batch_size, :],dtype=torch.float32)


In [ ]:
batch_size = 4096
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


input_size = 4*L # Define your input size based on the number of pixels in an image
output_size = 3  # Assuming RGB prediction

model_256_128_64_LBFGS =  MLP4(input_size,256,128,64, output_size)


start_time = time.time()


criterion = nn.MSELoss()
#optimizer = torch.optim.LBFGS(model_hd_LBFGS.parameters(), lr=1,max_eval=30,max_iter=3)
optimizer = torch.optim.LBFGS(model_256_128_64_LBFGS.parameters(), lr=0.01, max_iter=20, max_eval=None, tolerance_grad=1e-5, tolerance_change=1e-9, history_size=100)


def closure():
    optimizer.zero_grad()
    outputs = model_256_128_64_LBFGS(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    return loss


num_epochs =3
for epoch in tqdm(range(num_epochs)):
    for inputs, targets in train_dataloader:
        # Flatten the inputs if needed


        inputs = torch.tensor(inputs,dtype=torch.float32)
        targets = torch.tensor(targets,dtype=torch.float32)



        optimizer.step(closure)

    # Print the loss at each iteration
    print(f'Epoch {epoch + 1}, Loss: {closure().item()}')

print("---Duration: %s seconds ---" % (time.time() - start_time))


  0%|          | 0/3 [00:00<?, ?it/s]<ipython-input-27-e9ebb0450f5a>:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs,dtype=torch.float32)
<ipython-input-27-e9ebb0450f5a>:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets,dtype=torch.float32)
 33%|███▎      | 1/3 [18:22<36:44, 1102.17s/it]

Epoch 1, Loss: 0.02549625001847744


 67%|██████▋   | 2/3 [36:58<18:30, 1110.78s/it]

Epoch 2, Loss: 0.02551054023206234


100%|██████████| 3/3 [55:22<00:00, 1107.35s/it]

Epoch 3, Loss: 0.024893099442124367
---Duration: 3322.069444656372 seconds ---


In [ ]:
model_256_128_64_LBFGS.eval()
with torch.no_grad():
    test_loss = 0.0
    for inputs, targets in test_dataloader:
        # Flatten the inputs if needed

        inputs = torch.tensor(inputs,dtype=torch.float32)
        targets = torch.tensor(targets,dtype=torch.float32)

        outputs = model_256_128_64_LBFGS(inputs)

        test_loss += criterion(outputs, targets)

    average_test_loss = test_loss / len(test_dataloader)
    print(f'Test Loss: {average_test_loss:.4f}')

<ipython-input-28-94df87380c68>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs,dtype=torch.float32)
<ipython-input-28-94df87380c68>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets,dtype=torch.float32)


Test Loss: 0.0249


In [ ]:
torch.save(model_256_128_64_LBFGS, '/content/drive/My Drive/model_mlp_256_128_64_LBFGS.pt')

In [ ]:
coor = [dataset[i][0] for i in range(len(dataset))]
coor = torch.tensor(coor)

model_256_128_64_LBFGS.eval()

batch_size = 64
with torch.no_grad():
  pre = []
  for i in range(0, coor.size(0), batch_size):
    batch_coor = torch.tensor(coor[i:i+batch_size, :],dtype=torch.float32)
    batch_pred = model_256_128_64_LBFGS(batch_coor)
    pre.append(batch_pred)

predicted_rgb = torch.cat(pre, dim=0)

predicted_rgb = predicted_rgb.view(dataset.height, dataset.width, 3).numpy() * 255
predicted_image = Image.fromarray(predicted_rgb.astype('uint8'))

predicted_image.save('./mlp_256_128_64_LBFGS.png')

<ipython-input-30-335d0563e956>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_coor = torch.tensor(coor[i:i+batch_size, :],dtype=torch.float32)
